In [1]:
import psycopg2
from osgeo import ogr
from data import acces

In [2]:
connection = psycopg2.connect(dbname=acces.DATABASE_NAME,
                             user=acces.USER_NAME,
                             password=acces.PASS,
                             host='localhost',
                             port='5432')

In [3]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("DROP TABLE IF EXISTS borders")
        cursor.execute("CREATE TABLE borders (" +
                       "id SERIAL PRIMARY KEY," +
                       "name VARCHAR NOT NULL," +
                       "iso_code VARCHAR NOT NULL," +
                       "outline GEOGRAPHY)")
        cursor.execute("CREATE INDEX border_index ON borders " +
                       "USING GIST(outline)")
        connection.commit()

In [4]:
shapefile = ogr.Open('input/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp')
layer = shapefile.GetLayer(0)

In [5]:
for i in range(layer.GetFeatureCount()):
    feature = layer.GetFeature(i)
    name = feature.GetField('NAME')
    iso_code = feature.GetField('ISO3')
    geometry = feature.GetGeometryRef()
    wkt = geometry.ExportToWkt()
    with connection:
        with connection.cursor() as cursor:
            cursor.execute("INSERT INTO borders (name, iso_code, outline) " +
                           "VALUES (%s, %s, ST_GeogFromText(%s))",
                           (name, iso_code, wkt))
        connection.commit()

In [6]:
start_long = 8.542
start_lat = 47.377
radius = 500000

In [7]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("SELECT name FROM borders WHERE ST_DWithin(" +
                       "ST_MakePoint(%s, %s), outline, %s)",
                       (start_long, start_lat, radius))
        for row in cursor:
            print(row[0])

San Marino
Austria
Czech Republic
France
Germany
Croatia
Italy
Liechtenstein
Belgium
Luxembourg
Monaco
Netherlands
Slovenia
Switzerland


In [8]:
# обновляем оптимизатор запросов
with connection:
    with connection.cursor() as cursor:
        old_level = connection.isolation_level
        connection.set_isolation_level(0)
        cursor.execute("VACUUM ANALYZE") 
        connection.set_isolation_level(old_level)

In [9]:
# всегда надо закрывать коннект с базой 
# (он не закрывается в менеджере контекста)
connection.close()

In [10]:
connection = psycopg2.connect(host='localhost', 
                              port='5432', 
                              database=acces.DATABASE_NAME1, 
                              user=acces.USER_NAME1, 
                              password=acces.PASS1)
with connection:
    with connection.cursor() as cursor:
        cursor.execute('SELECT * FROM greeting')
        rows = cursor.fetchall() 
        for row in rows: 
            print(row[0])
connection.close()

Hello World!


In [ ]:
# Connect to your postgres DB
conn = psycopg.connect("dbname=test user=postgres")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("SELECT * FROM my_data")

# Retrieve query results
records = cur.fetchall()

[psycopg usage](https://www.psycopg.org/docs/usage.html)

The main entry points of Psycopg are:

The function connect() creates a new database session and returns a new connection instance.

The class connection encapsulates a database session. It allows to:
- create new cursor instances using the cursor() method to execute database commands and queries,
- terminate transactions using the methods commit() or rollback().

The class cursor allows interaction with the database:
- send commands to the database using methods such as execute() and executemany(),
- retrieve data from the database by iteration or using methods such as fetchone(), fetchmany(), fetchall().

Passing parameters to an SQL statement happens in functions such as cursor.execute() by using %s placeholders in the SQL statement, and passing a sequence of values as the second argument of the function. For example the Python function call:
```
>>> cur.execute("""
...     INSERT INTO some_table (an_int, a_date, a_string)
...     VALUES (%s, %s, %s);
...     """,
...     (10, datetime.date(2005, 11, 18), "O'Reilly"))
```

(!) Есть ряд нюансов с более сложными конструкциями и их синтаксисом. См. доку

**Warning Never, never, NEVER use Python string concatenation (+) or string parameters interpolation (%) to pass variables to a SQL query string. Not even at gunpoint.**

![](img/011.png)

**Адаптация констант**

```
>>> cur.mogrify("SELECT %s, %s, %s;", (None, True, False))
'SELECT NULL, true, false;'
```

**Адаптация чисел**

```
>>> cur.mogrify("SELECT %s, %s, %s, %s;", (10, 10L, 10.0, Decimal("10.00")))
'SELECT 10, 10, 10.0, 10.00;'
```

Reading from the database, integer types are converted into int, floating point types are converted into float, numeric/decimal are converted into Decimal.

**Адаптация строк**

Python str and unicode are converted into the SQL string syntax. unicode objects (str in Python 3) are encoded in the connection encoding before sending to the backend: trying to send a character not supported by the encoding will result in an error.

Другие адаптации и подробности см. в доке

**Транзауции**

The connection is responsible for terminating its transaction, calling either the commit() or rollback() method. Committed changes are immediately made persistent in the database. If the connection is closed (using the close() method) or destroyed (using del or by letting it fall out of scope) while a transaction is in progress, the server will discard the transaction. However doing so is not advisable: middleware such as PgBouncer may see the connection closed uncleanly and dispose of it.

Сonnections and cursors are context managers and can be used with the with statement:

In [ ]:
conn = psycopg2.connect(DSN)

with conn:
    with conn.cursor() as curs:
        curs.execute(SQL1)

with conn:
    with conn.cursor() as curs:
        curs.execute(SQL2)

conn.close()

**Warning Unlike file objects or other resources, exiting the connection’s with block doesn’t close the connection, but only the transaction associated to it. If you want to make sure the connection is closed after a certain point, you should still use a try-catch block:**

In [ ]:
conn = psycopg2.connect(DSN)
try:
    # connection usage
finally:
    conn.close()

- [psycopg2 методы](https://www.psycopg.org/docs/module.html)
- [connection class](https://www.psycopg.org/docs/connection.html)
- [cursor class](https://www.psycopg.org/docs/cursor.html)
- [advenced](https://www.psycopg.org/docs/advanced.html)
- остальное смотри в доке